In [2]:
import pandas as pd

In [2]:
df_Top1000Delegates = pd.read_csv("./Top1000Delegates.csv")
df_vote_cast = pd.read_csv("./vote_cast.csv")
df_proposals = pd.read_csv("./proposals.csv")

In [ ]:
df_proposals_votes = pd.read_csv("./raw_data/2024-06-28_optimism_proposals_votes.csv")

In [15]:
df_onchain = df_proposals_votes.query('vote_type == "ON_CHAIN"').reset_index(drop=True)

# Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
df_sorted = df_onchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# Step 2: Group by 'proposal_id' and select the top 200 for each group
df_top_200 = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)

df_top_200['reason'] = df_top_200['reason'].fillna('NA')

In [21]:
df_top_200.to_csv("./proposals_votes.csv")

In [18]:
df_sorted.groupby('proposal_id').head(3).reset_index(drop=True)

,address,ens_address,proposal_id,title,url,voted_at,vote_type,reason,voting_power
0,0x5e349eca2dc61abcd9dd99ce94d04136151a09ee,lindajxie.eth,Proposal|Proposal|1028219989334601591562635448...,Code of Conduct Council: Member Nominations,https://vote.optimism.io/proposals/10282199893...,2023-11-13 07:56:35,ON_CHAIN,NaN,5.968178e+06
1,0x429f9ada43e9f345cbb85ec88681bb70df808892,polynya.eth,Proposal|Proposal|1028219989334601591562635448...,Code of Conduct Council: Member Nominations,https://vote.optimism.io/proposals/10282199893...,2023-11-05 11:04:33,ON_CHAIN,NaN,5.203963e+06
2,0x406b607644c5d7bfda95963201e45a4c6ab1c159,NaN,Proposal|Proposal|1028219989334601591562635448...,Code of Conduct Council: Member Nominations,https://vote.optimism.io/proposals/10282199893...,2023-11-08 15:22:39,ON_CHAIN,NaN,3.431752e+06
3,0x406b607644c5d7bfda95963201e45a4c6ab1c159,NaN,Proposal|Proposal|1036064007985958030126449663...,NaN,https://vote.optimism.io/proposals/10360640079...,2023-02-13 14:05:10,ON_CHAIN,SNX Ambassadors Test,2.330198e+06
4,0x5e349eca2dc61abcd9dd99ce94d04136151a09ee,lindajxie.eth,Proposal|Proposal|1036064007985958030126449663...,NaN,https://vote.optimism.io/proposals/10360640079...,2023-02-14 18:46:00,ON_CHAIN,NaN,1.743039e+06
...,...,...,...,...,...,...,...,...,...
209,0x429f9ada43e9f345cbb85ec88681bb70df808892,polynya.eth,Proposal|Proposal|9686813595811107806498793885...,Grants Council Reviewer Elections: Milestones ...,https://vote.optimism.io/proposals/96868135958...,2023-11-07 06:31:39,ON_CHAIN,NaN,5.203965e+06
210,0x406b607644c5d7bfda95963201e45a4c6ab1c159,NaN,Proposal|Proposal|9686813595811107806498793885...,Grants Council Reviewer Elections: Milestones ...,https://vote.optimism.io/proposals/96868135958...,2023-11-08 15:22:39,ON_CHAIN,NaN,3.431549e+06
211,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,delegate.l2beat.eth,Proposal|Proposal|9846529187254295121420932271...,"Mission Requests: Intent #1, 1.33M OP",https://vote.optimism.io/proposals/98465291872...,2024-02-14 15:30:21,ON_CHAIN,https://gov.optimism.io/t/l2beat-delegate-comm...,4.931362e+06
212,0x5e349eca2dc61abcd9dd99ce94d04136151a09ee,lindajxie.eth,Proposal|Proposal|9846529187254295121420932271...,"Mission Requests: Intent #1, 1.33M OP",https://vote.optimism.io/proposals/98465291872...,2024-02-14 01:33:27,ON_CHAIN,NaN,4.856109e+06
